# **Experiment 6**

## **Aim: Multi agent in a search space**

## **Theory**

**Multi-Agent Search Theory (with Time-Step Coordination)**

Definition: Multi-agent search involves multiple agents navigating a shared environment to reach their individual goals without colliding with each other.

Key Challenge: Preventing conflicts (e.g., two agents occupying the same cell at the same time) while ensuring each agent finds a valid path.

**Time-Step Based Coordination**

Approach: Each agent plans its path sequentially. The first agent moves freely, and subsequent agents avoid the first agent’s location at each time step.

Blocking Strategy: A time-indexed map (e.g., {timestep: set of blocked cells}) is used to dynamically avoid positions that are temporarily occupied by other agents.

**Search Technique**

Breadth-First Search (BFS) is used with modifications to check time-based blocked positions, allowing dynamic and safe path planning for multiple agents.

## **Code & Output**

In [ ]:
from collections import deque

# Maze grid: 0 - free, 1 - wall
maze = [
    [0, 0, 1, 0, 0],
    [0, 1, 0, 1, 0],
    [0, 1, 0, 0, 0],
    [0, 0, 0, 1, 1],
    [0, 1, 0, 0, 0]
]

# Moves: up, down, left, right
moves = [(-1, 0), (1, 0), (0, -1), (0, 1)]

# Check if a position is within bounds and not a wall
def is_valid(maze, x, y):
    return 0 <= x < len(maze) and 0 <= y < len(maze[0]) and maze[x][y] == 0

# BFS with time-step based dynamic blocking
def bfs_with_dynamic_block(maze, start, goal, dynamic_blocks=None):
    queue = deque()
    visited = set()
    queue.append((start, [start], 0))  # (current_pos, path, timestep)

    while queue:
        (x, y), path, t = queue.popleft()

        if (x, y, t) in visited:
            continue
        visited.add((x, y, t))

        if (x, y) == goal:
            return path

        for dx, dy in moves:
            nx, ny = x + dx, y + dy
            if is_valid(maze, nx, ny):
                next_time = t + 1
                if dynamic_blocks and (nx, ny) in dynamic_blocks.get(next_time, set()):
                    continue
                queue.append(((nx, ny), path + [(nx, ny)], next_time))
    return None

# Agent A's path
def plan_agent_a(maze, start, goal):
    return bfs_with_dynamic_block(maze, start, goal)

# Build dynamic block dictionary from Agent A’s path
def build_dynamic_blocks(pathA):
    blocks = {}
    for t, pos in enumerate(pathA):
        blocks.setdefault(t, set()).add(pos)
    return blocks

# Main multi-agent coordination
def multi_agent_coordinated_search(maze, agentA, agentB):
    pathA = plan_agent_a(maze, agentA['start'], agentA['goal'])
    if not pathA:
        return None, None

    blocks = build_dynamic_blocks(pathA)
    pathB = bfs_with_dynamic_block(maze, agentB['start'], agentB['goal'], dynamic_blocks=blocks)

    return pathA, pathB

# Define agents
agentA = {'start': (0, 0), 'goal': (4, 4)}
agentB = {'start': (4, 0), 'goal': (0, 4)}

# Run search
pathA, pathB = multi_agent_coordinated_search(maze, agentA, agentB)

# Output paths
print("🚶 Agent A Path:", pathA)
print("🚶 Agent B Path:", pathB)

🚶 Agent A Path: [(0, 0), (1, 0), (2, 0), (3, 0), (3, 1), (3, 2), (4, 2), (4, 3), (4, 4)]
🚶 Agent B Path: [(4, 0), (3, 0), (3, 1), (3, 2), (2, 2), (2, 3), (2, 4), (1, 4), (0, 4)]
